In [209]:
## Run when initialise the code
import os

import geopandas as gpd
import osmnx as ox
from geopandas import GeoDataFrame, GeoSeries
from osmnx import io
import glob

project_crs = 'epsg:3857'
from sklearn.cluster import DBSCAN
from shapely.geometry import Polygon, Point, LineString, MultiPolygon, MultiPoint
import math
import warnings
import pandas as pd
import shutil
from tqdm import tqdm
import time
warnings.filterwarnings(action='ignore')
from momepy import remove_false_nodes,extend_lines
pjr_loc = os.path.dirname(os.getcwd())



<span style="color: Green;font-size: 30px">Module 1:Preliminary work</span>
<ul> <li>Download data from OpenStreetMap, project it, and convert it to a GeoDataFrame. OSMnx automatically resolves topology errors and retrieves only the street-related polylines.</li>
 <li>Identify roundabout elements, if any exist, and store them in a separate DataFrame.</li>
  <li>Remove additional irrelevant line objects based on values of the OSM 'tunnel' and 'highway' keys.</li>
   <li>Eliminate polylines that lack a name and calculate angles ranging from 0 to 180 degrees based on the bearing field.</li>
   </ul>


<span style="color: Red;font-size: 30px">Module 2 -3:Detect  parallel streets segments and merge them </span>
<ul> <li>For each group of streets with the same name search for parallel segments</li>
 <li> Use DBSCAN to classify streets based on their angle, and group each class. Outliers could not consider parallel with any street, thus removed</li>
  <li>The parallel test is on street segments that  have the same name and belong to the same angle group.
    <ul><li>Eliminate polylines that lack a name and calculate angles ranging from 0 to 180 degrees based on the bearing field.</li></ul>
    </li>

   </ul>

In [210]:
## Run when initialise the code
# In this example, the data is extracted from OSM by specifying a location's name, but you can also download data using a specified polygon. The code is designed to handle multiple polygons or location names seamlessly.

# Download data from OpenStreetMap, project it, and convert it to a GeoDataFrame. OSMnx automatically resolves topology errors and retrieves only the street-related polylines.

place = 'Turin'
print(place)
data_folder  = f'places/{place.replace(",","_").replace(" ","_")}_test'
os.makedirs(f'{data_folder}/delete_2_nodes',exist_ok = True)
os.makedirs(f'{data_folder}/split_tp_intersection',exist_ok = True)

Turin


In [204]:
if place =='Tel Aviv':
    useful_tags_path = ['name:en','highway','length','bearing','tunnel','junction']
    ox.utils.config(useful_tags_way=useful_tags_path)

In [205]:
data_folder

'places/Tel_Aviv_test'

In [206]:
graph = ox.graph_from_place(place, network_type='all')
graph = ox.bearing.add_edge_bearings(graph, precision=1)
graph_pro = ox.projection.project_graph(graph, to_crs=project_crs)
io.save_graph_geopackage(graph_pro, filepath=f'{data_folder}/osm_data.gpkg', encoding='utf-8', directed=False)


In [394]:
my_gdf = gpd.read_file(f'{data_folder}/osm_data.gpkg',layer = 'edges')# Identify roundabout elements, if any exist, and store them in a separate DataFrame.
if place =='Tel Aviv':
    my_gdf.rename(columns={'name:en':'name'}, inplace=True)
my_gdf

,u,v,key,osmid,oneway,ref,highway,maxspeed,reversed,length,...,to,name,lanes,access,bridge,junction,width,service,tunnel,geometry
0,5654262,305690687,0,23444821,True,A55,motorway_link,40,False,193.072,...,305690687,,,,,,,,,"LINESTRING (858869.813 5642733.262, 858816.357..."
1,5654262,317534008,0,150093791,True,,motorway_link,40,False,416.876,...,317534008,,,,,,,,,"LINESTRING (858869.813 5642733.262, 858811.281..."
2,5654262,1630170922,0,303020335,True,A55,motorway,130,False,15.200,...,5654262,Raccordo della Falchera,2,,,,,,,"LINESTRING (858886.756 5642746.612, 858869.813..."
3,305690687,305690672,0,45061425,True,,motorway_link,40,False,87.798,...,305690672,,,,,,,,,"LINESTRING (858647.742 5642577.073, 858768.000..."
4,317534008,317534010,0,659782179,True,A55,motorway,110,False,439.954,...,317534008,Tangenziale Nord,2,,,,,,,"LINESTRING (858090.432 5642519.918, 858650.591..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49753,11158697340,11158697321,0,1203895129,False,,footway,,True,17.149,...,11158697340,,,,,,,,,"LINESTRING (852528.765 5632217.782, 852507.002..."
49754,11158697340,11158697339,0,1203895146,False,,service,,True,7.005,...,11158697340,,,,,,,,,"LINESTRING (852502.305 5632219.862, 852503.629..."
49755,11158664093,11158664096,0,44104491,True,,residential,50,False,41.485,...,11158664096,Via Nino Bixio,1,,,,,,,"LINESTRING (852272.140 5632174.232, 852245.223..."
49756,11158664102,11158664104,0,1203895138,False,,footway,,True,9.738,...,11158664102,,,,,,,,,"LINESTRING (852427.554 5632013.292, 852421.309..."


In [395]:
## Run when initialise the code
# find and store roundabout
my_gdf = gpd.read_file(f'{data_folder}/osm_data.gpkg',layer = 'edges')# Identify roundabout elements, if any exist, and store them in a separate DataFrame.
is_junction= True if 'junction' in my_gdf.columns else False
if is_junction:
    round_about = my_gdf[my_gdf['junction'].isin(['roundabout', 'circular'])]
    my_gdf= my_gdf[~((my_gdf['junction'] == 'roundabout') | (my_gdf['junction'] == 'circular'))]

In [396]:

# Remove additional irrelevant line objects based on values of the OSM 'tunnel' and 'highway' keys.
if 'tunnel' in my_gdf.columns:
    my_gdf = my_gdf[~((my_gdf['tunnel'] == 'building_passage') | (my_gdf['tunnel'] == 'yes'))]
to_remove = my_gdf[~((my_gdf['highway'] == 'motorway') | (my_gdf['highway'] == 'trunk')| (my_gdf['highway'] == 'motorway_link')| (my_gdf['highway'] == 'motorway_link')| (my_gdf['highway'] == 'trunk_link'))]


# Eliminate polylines that lack a name and calculate angles ranging from 0 to 180 degrees based on the bearing field.
df_pro = to_remove.to_crs(project_crs).dropna(subset=['name'])
df_pro = df_pro[df_pro['name']!='']
df_pro['angle'] = df_pro['bearing'].apply(lambda x: x if x < 180 else x - 180)
df_pro['length'] = df_pro.length

In [398]:
df_pro['name_2'] = df_pro['name'].apply(lambda x:x[0] if isinstance(x,list) else x)


5         Corso Principe Oddone
6                Piazza Statuto
7        Corso Principe Eugenio
8                Piazza Statuto
9         Corso Principe Oddone
                  ...          
49737             Corso Venezia
49739             Corso Venezia
49741             Corso Venezia
49743               Via Breglio
49755            Via Nino Bixio
Name: name_2, Length: 27000, dtype: object

In [399]:
df_pro

,u,v,key,osmid,oneway,ref,highway,maxspeed,reversed,length,...,lanes,access,bridge,junction,width,service,tunnel,geometry,angle,name_2
5,13924308,945268794,0,61293337,True,,tertiary,,False,14.765415,...,,,,,,,,"LINESTRING (853760.994 5633752.261, 853762.875...",7.3,Corso Principe Oddone
6,13924308,1625774856,0,133270821,True,,primary,,False,29.481956,...,2,,,,,,,"LINESTRING (853760.994 5633752.261, 853731.528...",91.9,Piazza Statuto
7,13924308,6711150268,0,125743679,True,,primary,,False,25.112347,...,2,,,,,,,"LINESTRING (853785.084 5633759.355, 853760.994...",73.6,Corso Principe Eugenio
8,13924308,4738859081,0,966240517,True,,primary,,False,14.741098,...,2,,,,,,,"LINESTRING (853760.282 5633737.537, 853760.994...",2.8,Piazza Statuto
9,945268794,4612216359,0,186768349,True,,tertiary,,False,25.535312,...,,,,,,,,"LINESTRING (853762.875 5633766.906, 853774.130...",26.2,Corso Principe Oddone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49737,11152626706,11152626738,0,"[555010500, 1020865228]",True,,primary,,False,30.434909,...,2,yes,,,,,,"LINESTRING (855579.721 5637034.031, 855572.730...",35.5,Corso Venezia
49739,11152566893,11152626707,0,1020865229,True,,primary,,False,22.518760,...,,,,,,,,"LINESTRING (855594.749 5637018.434, 855608.141...",36.5,Corso Venezia
49741,11152566901,11152626712,0,1033056948,True,,residential,,False,16.673466,...,,,,,,,,"LINESTRING (855596.986 5636992.382, 855601.072...",37.1,Corso Venezia
49743,11152626709,11152626708,0,"[1203064155, 28105724]",True,,tertiary,,False,13.133241,...,,,,,,,,"LINESTRING (855558.681 5637036.759, 855551.201...",97.4,Via Breglio


In [81]:
# region
# Functions and classes to be utilized - Module 2
def length_of_parallel(my_s_join: GeoDataFrame, the_buffer: GeoSeries, geo_field: str) -> int:
    my_s_join['geometry'] = my_s_join[geo_field]
    new_data_0 = my_s_join.sjoin(GeoDataFrame(geometry=the_buffer, crs=project_crs), how='inner').reset_index()
    if len(new_data_0) == 0:
        return 0
    return len(new_data_0[new_data_0['index'] != new_data_0['index_right']])
def check_parallelism(to_translate: GeoDataFrame) -> bool:
    my_buffer = to_translate['geometry'].buffer(cap_style=2, distance=30, join_style=3)
    to_translate['geometry_right'] = to_translate['geometry'].apply(lambda x: x.parallel_offset(35, 'right'))
    to_translate['geometry_left'] = to_translate['geometry'].apply(lambda x: x.parallel_offset(35, 'left'))
    if length_of_parallel(to_translate, my_buffer, 'geometry_right') > 10 or length_of_parallel(to_translate, my_buffer, 'geometry_left') > 10:
        return True
    else:
        return False

# Functions and classes to be utilized - Module 3
def update_list(line_local):
    """
    add the first start/end point into the list
    :param line_local:
    :return:
    """
    list_pnts_of_line_group.extend([Point(line_local.coords[0]), Point(line_local.coords[-1])])
def create_center_line(one_poly):
    """
    This method calculate new line between the farthest points of the simplified polygon
    :param one_poly:
    :return:
    """

    pnt_list = one_poly.exterior.coords[:-1]
    list_shp = [Point(item) for item in pnt_list]
    dis, dis_2, dis_3 = 0, 0, 0
    third_dis = (-1, -1)
    # The new line will be determined by the third-farthest points
    for k, point in enumerate(list_shp):
        j = k
        for point2 in list_shp[k + 1:]:
            j += 1
            temp_dis = point.distance(point2)
            if temp_dis > dis:
                dis = point.distance(point2)
            elif temp_dis > dis_2:
                dis_2 = point.distance(point2)
            elif temp_dis > dis_3:
                third_dis = (k, j)
                dis_3 = point.distance(point2)
    max_dist['name'].extend([id_pol + 2, id_pol + 2])
    max_dist['geometry'].extend([list_shp[third_dis[0]], list_shp[third_dis[1]]])
def add_more_pnts_to_new_lines(pnt_f_loc: Point, pnt_l_loc: Point, line_pnts: list) -> list:
    """
    This method checks if more points should be added to the new lines by checking along the new line if the distance to the old network roads are more than 10 meters
    :return:
    """
    # Calculate distance and azimuth between the first and last point
    dist = pnt_f_loc.distance(pnt_l_loc)
    x_0 = pnt_f_loc.coords[0][0]
    y_0 = pnt_f_loc.coords[0][1]
    bearing = math.atan2(pnt_l_loc.coords[0][0] - x_0, pnt_l_loc.coords[0][1] - y_0)
    bearing = bearing + 2 * math.pi if bearing < 0 else bearing
    # Calculate the number of  checks going to carry out
    length_to_check  =50
    loops = int(dist /length_to_check)

    # Calculate  the first point over the line
    for dis_on_line in range(1, loops):
        x_new = x_0 + length_to_check  * dis_on_line * math.sin(bearing)
        y_new = y_0 + length_to_check  * dis_on_line * math.cos(bearing)
        # S_joins to all the network lines (same name and group)
        # if the distance is less than 10 meters continue, else: find the projection point and add it to the correct location and run the function agein
        one_pnt_df = GeoDataFrame(geometry=[Point(x_new, y_new)], crs=project_crs)
        s_join_loc = one_pnt_df.sjoin_nearest(data, distance_col='dis').iloc[0]
        if s_join_loc['dis'] > 10:
            pnt_med = s_join_loc['geometry']
            line = data.loc[s_join_loc['index_right']]['geometry']
            line_pnts.append(line.interpolate(line.project(pnt_med)))
            line_pnts = add_more_pnts_to_new_lines(pnt_med, pnt_l_loc, line_pnts)
            return line_pnts
    return line_pnts
def update_df_with_center_line(new_line,is_simplified=0,group_name= -1):
    """
    update our dictionary with new lines
    :param is_simplified:
    :param new_line:
    :param group_name: According to the DBSCAN algorithm, if no =-1
    :return:
    """
    dic_final['name'].append(name)
    # dic_final['geometry'].append(LineString(coordinates=(pnt_list[max_dis[0]], pnt_list[max_dis[1]])))
    dic_final['geometry'].append(new_line)
    dic_final['highway'].append(data.iloc[0]['highway'])
    dic_final['bearing'].append(data['angle'].mean())
    dic_final['group'].append(group_name)
    dic_final['is_simplified'].append(is_simplified)
# group the street segments by street name
my_groupby = df_pro.groupby('name')
dic_final = {'name': [], 'geometry': [], 'highway': [], 'bearing': [], 'group': [],'is_simplified':[]}

for_time = len(my_groupby)
# endregion

In [82]:
is_test = False
number_of_parallel = 0 # count the number of polylines were refined
with tqdm(total=for_time) as pbar: #  It is used in order to visualise the progress by progress bar
    for i, street in enumerate(my_groupby):
        if is_test:
            name = 'Corso Orbassano','Corso Umbria',"['Corso Venezia', 'Raccordo Autostradale Torino-Caselle'].'monterey, boulevard, the street for example,Corso Achille Mario Dogliotti, Vicolo Valtorta IN THE END"
            res = my_groupby.get_group(name)
        else:
            res = street[1] # it holds all the streets
            name = street[0] # It holds the streets name
                # For each group of streets with the same name search for parallel segments
        pbar.update(1) # for the progress bar
        # Remove segments without angle. If less than two segments being left move to the next group.
        res = res.dropna(subset=['angle'], axis=0)
        if len(res) < 2:
            data = res
            _  = res['geometry'].apply(lambda x:update_df_with_center_line(x))
            continue
        # Use DBSCAN to classify streets based on their angle, and group each class. Outliers could not consider parallel with any street, thus removed
        res['group'] = DBSCAN(eps=10, min_samples=2).fit(res['angle'].to_numpy().reshape(-1, 1)).labels_
        # if all is -1, don't touch the element
        if (res['group']== -1).all():
            data = res
            _  = res['geometry'].apply(lambda x:update_df_with_center_line(x))
            continue
        # cur_group = res[(res['group'] > -1) | (res.length>20)].groupby('group') # Remove short segments with -1 classification values
        # The parallel test is on street segments that  have the same name and belong to the same angle group.
        for group in res.groupby('group'):
            data = group[1]
            if group[0] ==-1: # No need to check if is parallel
                _  = data['geometry'].apply(lambda x:update_df_with_center_line(x))
                continue
            if check_parallelism(data.copy()):
                number_of_parallel+=len(data) # Update the number of parallel polylines
                # if among of lines with same angles some are parallel:
                # new points DataFrame of start/end line of each group
                list_pnts_of_line_group = []
                data['geometry'].apply(update_list)
                df_pnts = GeoDataFrame(geometry=list_pnts_of_line_group, crs=project_crs).drop_duplicates()

                # unify lines to one polygon
                buffers = data.buffer(cap_style=3, distance=30, join_style=3)
                one_buffer = buffers.unary_union

                max_dist = {'name': [], 'geometry': []}
                # simplify polygon with simplify function. If one_buffer is multipolygon object simplify each one them separately
                if isinstance(one_buffer, MultiPolygon):
                    for id_pol, polygon in enumerate(one_buffer):
                        create_center_line(polygon)
                else:
                    id_pol = -1
                    create_center_line(one_buffer)
                max_df = GeoDataFrame(max_dist, crs=project_crs)
                max_df.to_file(f'{data_folder}/max_df.shp')
                # find for each points the closet point from the oribinal data. the closet points will create the new line
                s_join = max_df.sjoin_nearest(df_pnts).groupby('name')
                for geo in s_join:
                    same_name = geo[1]
                    if same_name.iloc[0]['index_right'] == same_name.iloc[1]['index_right']:
                        continue
                    in_0 = same_name.iloc[0]['index_right']
                    in_1 = same_name.iloc[1]['index_right']
                    # These points will be served to be initial reference in order to find more points
                    pnt_f = df_pnts.loc[in_0]['geometry']
                    pnt_l = df_pnts.loc[in_1]['geometry']
                    lines_pnt_geo = add_more_pnts_to_new_lines(pnt_f, pnt_l, [pnt_f])
                    lines_pnt_geo.append(pnt_l)
                    # Update dic_final
                    update_df_with_center_line(LineString(lines_pnt_geo),1,group[0])

            else:
                _  = data['geometry'].apply(lambda x:update_df_with_center_line(x))
        if is_test:
            break


print(number_of_parallel)

100%|██████████| 2493/2493 [03:52<00:00, 10.70it/s] 

11133


In [89]:
print('create new files')
# remove short lines
final_cols = ['name', 'geometry', 'highway', 'bearing', 'length']
new_network = GeoDataFrame(dic_final, crs=project_crs)

# create network
new_network.to_file(f'{data_folder}/simp.shp')

create new files


In [214]:
# starting point
new_network = gpd.read_file(f'{data_folder}/simp.shp')
new_network

,name,highway,bearing,group,is_simplif,geometry
0,11,path,106.60,-1,0,"LINESTRING (857888.020 5626761.570, 857902.213..."
1,11,path,106.60,-1,0,"LINESTRING (857638.976 5626560.639, 857618.671..."
2,18,path,40.85,-1,0,"LINESTRING (860173.665 5627099.066, 860141.427..."
3,18,path,40.85,-1,0,"LINESTRING (860173.665 5627099.066, 860196.674..."
4,18,path,164.90,-1,0,"LINESTRING (860110.012 5627377.578, 860111.003..."
...,...,...,...,...,...,...
16264,"['Viale Enrico Millo', 'Viale Virgilio']",pedestrian,7.40,-1,0,"LINESTRING (855537.353 5629215.626, 855536.028..."
16265,"['Viale Rombon', ""Viale Stretta D'Auzza""]",path,84.20,-1,0,"LINESTRING (859391.734 5626943.304, 859404.759..."
16266,"['Viale San Gabriele', 'Viale Castelgomberto']",path,30.80,-1,0,"LINESTRING (858361.706 5626215.887, 858343.194..."
16267,"['Viale Stefano Turr', 'Viale Virgilio']",pedestrian,27.10,-1,0,"LINESTRING (855542.740 5629433.523, 855532.020..."


In [359]:
# region
# Classes to be employed during the execution of this code.
#Intersection
#Split in intersection
class Intersection:
    def __init__(self,network:GeoDataFrame,number:int):
        """

        :param network:
        :param number: give a unique name to the files created during the process (this class will be use again in this code)
        """
        self.my_network = network
        self.inter_pnt_dic = {'geometry':[],'name':[]}
        self.lines_to_delete =[]
        self.num = number
    def delete_false_intersection(self,name_to_splt='name'):

        if 'length' in self.my_network.columns: # To run the code smoothly we need to remove 'length' col if exist
            self.my_network.drop(columns='length',inplace= True)

        # It should be executed twice in order to clean all
        for _  in range(2):
            # First clean all the false node
            self.my_network = remove_false_nodes(self.my_network)
            # the previous function has changed the topology so the length should be updated
            self.my_network['length'] =self.my_network.length
            self.my_network  =self.my_network.drop_duplicates(subset='length') # remove false intersection duplicate many polyline which should be removed
            self.my_network.reset_index(drop=True,inplace= True) # Changes has been made to the geometry so the index should be reset





    def intersection_network(self):

        # Create buffer around each element
        buffer_around_lines= self.my_network['geometry'].buffer(cap_style=3, distance=1, join_style=3)


        # s_join between buffer to lines
        s_join_0 =gpd.sjoin(left_df=GeoDataFrame(geometry=buffer_around_lines,crs=project_crs),right_df=self.my_network)

        # delete lines belong to the buffer
        s_join = s_join_0[s_join_0.index!=s_join_0['index_right']]


        # Find new intersections that are not at the beginning or end of the line
        for_time =len(s_join)
        with tqdm(total=for_time) as pbar:
            s_join.apply(lambda x: self.find_intersection_points(x,pbar), axis=1)
        if len(self.inter_pnt_dic)==0:
            return
        inter_pnt_gdf = GeoDataFrame(self.inter_pnt_dic,crs=project_crs)

        # Split string line by points
        segments = {'geometry':[],'org_id':[]}
        # Groupby points name (which is the line they should split)
        my_groups =  inter_pnt_gdf.groupby('name')
        for_time = len(my_groups)
        with  tqdm(total=for_time) as pbar:
            for group_pnts in my_groups :
                pbar.update(1)
                points  = group_pnts[1]
                points['is_split'] = True

                # get the line to split by comparing the name
                row = self.my_network.loc[group_pnts[0]]
                current = list(row.geometry.coords)
                points_line = [Point(x) for x in current]
                points_line_gdf = GeoDataFrame(geometry=points_line,crs=project_crs)
                points_line_gdf['is_split'] = False

                # append all the points together (line points and split points)
                line_all_pnts = points_line_gdf.append(points)

                # Find the distance of each point form the begining of the line on the line.
                line_all_pnts['dis_from_the_start'] = line_all_pnts['geometry'].apply(lambda x:row.geometry.project(x))
                line_all_pnts.sort_values('dis_from_the_start',inplace=True)

                # split the line
                seg =[]
                for point in line_all_pnts.iterrows():
                    prop = point[1]
                    seg.append(prop['geometry'])
                    if prop['is_split']:
                        segments['geometry'].append(LineString(seg))
                        segments['org_id'].append(row.name)
                        seg = [prop['geometry']]
                # if the split point is the last one, you don't need to create new segment
                if len(seg)>1:
                    segments['geometry'].append(LineString(seg))
                    segments['org_id'].append(row.name)
        network_split = GeoDataFrame(data=segments,crs=project_crs)
        cols_no_geometry = self.my_network.columns[:-1]
        network_split_final = network_split.set_index('org_id')
        network_split_final[cols_no_geometry] =self.my_network[cols_no_geometry]

        # remove old and redundant line from our network and update with new one
        network_split =self.my_network.drop(index=network_split_final.index.unique()).append(network_split_final).drop(index= self.lines_to_delete)
        network_split['length'] = network_split.length
        self.my_network = network_split
        self.my_network.reset_index(drop=True,inplace= True)

    def find_intersection_points(self,row,pbar):
        r"""
        find the intersection points between the two lines
        :param row:
        :return:
        """
        try:
            pbar.update(1)
            line_1 = self.my_network.loc[row.name]
            line_2 =  self.my_network.loc[row['index_right']]
            pnt = line_1.geometry.intersection(line_2.geometry)
            # If there are more than one intersection between two lines, one of the lines should be deleted.
            if isinstance(pnt,LineString):
                return
            if isinstance(pnt,MultiPoint):
                temp_line= line_1.name if line_1.length< line_2.length else line_2.name
                if temp_line not in self.lines_to_delete:
                    self.lines_to_delete.append(temp_line)
                return
            # If it is first or end continue OR if there is no intersection between the two lines
            if len(pnt.coords)==0 or pnt.coords[0]==line_1.geometry.coords[0] or pnt.coords[0]==line_1.geometry.coords[-1]:
                return
            self.inter_pnt_dic['geometry'].append(pnt)
            self.inter_pnt_dic['name'].append(row.name)
        except:
            print(f"{row.name},{row['index_right']}:{pnt}")


#Roundabout
class EnvEntity:
        def __init__(self,network):
            self.dead_end_fd = None
            self.pnt_dead_end = None
            self.pnt_dic = {}
            self.first_last_dic = {'geometry': [], 'line_name': [], 'position': []}
            self.network = network


        def __populate_pnt_dic(self,point: type, name_of_line: str):
            """
            Make "pnt_dic" contain a list of all the lines connected to each point.
            :param point:
            :param name_of_line:
            :return:
            """
            if not point in self.pnt_dic:
                self.pnt_dic[point] = []
            self.pnt_dic[point].append(name_of_line)

        def __send_pnts(self,temp_line: GeoSeries):
            """
            # Send the first and the last points to populate_pnt_dic
            :return:
            """
            my_geom = temp_line['geometry']
            self.__populate_pnt_dic(my_geom.coords[0], temp_line.name)
            self.__populate_pnt_dic(my_geom.coords[-1], temp_line.name)

        def get_deadend_gdf(self,delete_short:int =30)-> GeoDataFrame:
            self.network.apply(self.__send_pnts, axis=1)

            deadend_list = [item[1][0] for item in self.pnt_dic.items() if len(item[1]) == 1]
            pnt_dead_end_0 = [item for item in self.pnt_dic.items() if len(item[1]) == 1] # Retain all the line points with deadened
            self.pnt_dead_end = [Point(x[0]) for x in pnt_dead_end_0]
            # Create shp file of deadened_pnts
            geometry,line_name = 'geometry','line_name'
            pnt_dead_end_df = GeoDataFrame(data=pnt_dead_end_0)
            pnt_dead_end_df[geometry]= pnt_dead_end_df[0].apply(lambda x:Point(x))
            pnt_dead_end_df[line_name] = pnt_dead_end_df[1].apply(lambda x:x[0])
            pnt_dead_end_df.crs = project_crs
            self.dead_end_fd = pnt_dead_end_df

            if delete_short>0:
                # If it is necessary to eliminate dead-end short segments, it is  important to delete them from the network geodataframe.

                deadend_gdf =self.network.loc[deadend_list]
                self.network.drop(index=deadend_gdf[deadend_gdf.length<delete_short].index,inplace=True)
                return deadend_gdf[deadend_gdf.length>delete_short]
            return self.network.loc[deadend_list]

        def update_the_current_network(self,temp_network):
            r"""
            Update the current network in the new changes
            :param temp_network:
            :return:
            """
            new_network_temp = self.network.drop(index=temp_network.index)
            self.network = new_network_temp.append(temp_network)
            self.network['length'] = self.network.length
            self.network  = self.network[self.network['length']>1]
class Roundabout(EnvEntity):
    def __init__(self,network: GeoDataFrame):
       EnvEntity.__init__(self,network)
       self.pnt_dic ={}
       self.centroid =self.__from_roundabout_to_centroid()
       self.network.rename(columns={'name': 'str_name'}, inplace=True)
    def __from_roundabout_to_centroid(self):
        # Find the center of each roundabout
        # create polygon around each polygon and union
        round_about_buffer = round_about.to_crs(project_crs)['geometry'].buffer(cap_style=1, distance=10,
                                                                                join_style=1).unary_union
        dic_data = {'name': [], 'geometry': []}
        if round_about_buffer.type=='Polygon': # In case we have only one polygon
            dic_data['name'].append(0)
            dic_data['geometry'].append(round_about_buffer.centroid)
        else:
            for ii, xx in enumerate(round_about_buffer):
                dic_data['name'].append(ii)
                dic_data['geometry'].append(xx.centroid)
        centroid =GeoDataFrame(dic_data, crs=project_crs)
        return centroid
        # GeoDataFrame(dic_data,crs=project_crs).to_file(f'{path_round_about}/roundabout_union.shp')

    def __first_last_pnt_of_line(self,row: GeoSeries):
        r"""
        It get geometry of line and fill the first_last_dic with the first and last point and the name of the line
        :return:
        """
        geo = list(row['geometry'].coords)
        self.first_last_dic['geometry'].extend([Point(geo[0]), Point(geo[-1])])
        self.first_last_dic['line_name'].extend([row.name] * 2)
        self.first_last_dic['position'].extend([0, -1])
    def deadend(self):
        r"""
        remove not connected line shorter than 100 meters and then return deadend_list lines and their endpoints (as another file)
        :return:
        """
        # Find the first and last points

        # Get deadend_gdf
        deadend_gdf = self.get_deadend_gdf()

        # Create gdf of line points with the reference to the line they belong
        deadend_gdf.apply(self.__first_last_pnt_of_line, axis=1)
        first_last_gdf = GeoDataFrame(self.first_last_dic, crs=project_crs)


        return deadend_gdf, first_last_gdf
    def __update_geometry(self,cur,s_join):
        r"""
        :return:
        """
        if cur['highway'] == 'footway':
            # Don't snap footway to roundabout
            return cur['geometry']
        # Get only the points that are deadened
        points_lines = [item for item in s_join[s_join['line_name'] == cur.name].iterrows()if item[1]['geometry'] in self.pnt_dead_end]
        if len(points_lines) == 0:
            # No roundabout nearby
            return cur['geometry']
        # get the line geometry to change the first and/ or last point
        geo_cur = list(cur['geometry'].coords)

        # iterate over the deadened points  near roundabout
        for ind in range(len(points_lines)):
            points_line = points_lines[ind]
            geo_cur[points_line[1]['position']] = self.centroid.loc[points_line[1]['index_right']]['geometry'].coords[
                0]
        return LineString(geo_cur)
    def my_spatial_join(self,deadend_lines, deadend_pnts,line_name):
        # Spatial join between roundabout centroid to nearby dead end lines
        # centroid = gpd.read_file(f'{path_round_about}/centroid.shp')
        s_join = gpd.sjoin_nearest(left_df=deadend_pnts, right_df=self.centroid, how='left', max_distance=100,
                                   distance_col='dist').dropna(subset='dist')

        # Deadened lines from both lines should be removed
        lines_to_delete_test = s_join['line_name'].unique() # all the Deadened lines close to roundabout

        # All deadened lines from both lines
        deads_both_side = self.dead_end_fd['line_name'].value_counts()
        deads_both_side =deads_both_side[deads_both_side==2]

        # Remove this lines from the database
        lines_to_delete=deads_both_side[deads_both_side.index.isin(lines_to_delete_test)]

        self.network = self.network[~((self.network[line_name].isin(lines_to_delete.index)) & (self.network.length<300))]
        deadend_lines = deadend_lines[~((deadend_lines[line_name].isin(lines_to_delete.index)) & (deadend_lines.length<300))]
        # Update the geometry so the roundabout will be part of the line geometry
        change_geo = deadend_lines.copy()

        change_geo['geometry'] = change_geo.apply(lambda x:self.__update_geometry(x,s_join), axis=1)

        return change_geo
# endregion

In [371]:

num=0
new_gpd = new_network.copy()
obj_intersection = Intersection(new_gpd,num)
obj_intersection.delete_false_intersection()
obj_intersection.intersection_network()

100%|██████████| 1403/1403 [00:07<00:00, 182.01it/s]


In [375]:
df1 = obj_intersection.my_network

In [393]:

new_gpd.reset_index().to_file(f'{data_folder}/delete_2_nodes/source.shp')

In [376]:
# Split df1 into two GeoDataFrames: df3 (with names) and df4 (without names)
df3 = df1[df1['name'].notna()]
df3.to_file(f'{data_folder}/delete_2_nodes/with_name.shp')


In [377]:
df4 = df1[df1['name'].isna()]
df4.reset_index().to_file(f'{data_folder}/delete_2_nodes/no_name_init.shp')

In [378]:
df4

,name,highway,bearing,group,is_simplif,length,geometry
6367,NaN,NaN,NaN,NaN,NaN,105.790786,"LINESTRING (845229.980 5628010.626, 845209.653..."
6368,NaN,NaN,NaN,NaN,NaN,1211.278442,"LINESTRING (845261.317 5627984.379, 845272.326..."
6369,NaN,NaN,NaN,NaN,NaN,924.639232,"LINESTRING (845543.289 5626651.149, 845800.493..."
6370,NaN,NaN,NaN,NaN,NaN,37.664672,"LINESTRING (846260.020 5626359.917, 846268.413..."
6371,NaN,NaN,NaN,NaN,NaN,110.627927,"LINESTRING (846279.144 5627190.684, 846287.927..."
...,...,...,...,...,...,...,...
12940,NaN,NaN,NaN,NaN,NaN,61.978956,"LINESTRING (860169.212 5633326.618, 860182.737..."
12941,NaN,NaN,NaN,NaN,NaN,74.589004,"LINESTRING (860185.722 5633386.276, 860185.743..."
12942,NaN,NaN,NaN,NaN,NaN,195.649495,"LINESTRING (860273.351 5635961.919, 860319.282..."
12943,NaN,NaN,NaN,NaN,NaN,13.416175,"LINESTRING (860386.753 5636096.233, 860398.913..."


In [379]:
# use only one polyline from the original dataframe for name even if the algorithm may found more
old_index  ='old_index'

df = gpd.sjoin(df4,new_gpd).reset_index(names='old_index')
df

,old_index,name_left,highway_left,bearing_left,group_left,is_simplif_left,length,geometry,index_right,name_right,highway_right,bearing_right,group_right,is_simplif_right
0,6367,NaN,NaN,NaN,NaN,NaN,105.790786,"LINESTRING (845229.980 5628010.626, 845209.653...",12013,Via Paolo Gorini,service,123.950000,-1,0
1,6367,NaN,NaN,NaN,NaN,NaN,105.790786,"LINESTRING (845229.980 5628010.626, 845209.653...",12012,Via Paolo Gorini,service,123.950000,-1,0
2,6368,NaN,NaN,NaN,NaN,NaN,1211.278442,"LINESTRING (845261.317 5627984.379, 845272.326...",11909,Via Pancalieri,unclassified,136.600000,-1,0
3,6372,NaN,NaN,NaN,NaN,NaN,78.701774,"LINESTRING (846279.222 5627331.531, 846280.491...",11909,Via Pancalieri,unclassified,136.600000,-1,0
4,10275,NaN,NaN,NaN,NaN,NaN,94.032923,"LINESTRING (846279.222 5627331.531, 846279.011...",11909,Via Pancalieri,unclassified,136.600000,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24412,12940,NaN,NaN,NaN,NaN,NaN,61.978956,"LINESTRING (860169.212 5633326.618, 860182.737...",1895,Lungo Po Alessandro Antonelli,residential,27.653333,-1,0
24413,12941,NaN,NaN,NaN,NaN,NaN,74.589004,"LINESTRING (860185.722 5633386.276, 860185.743...",1895,Lungo Po Alessandro Antonelli,residential,27.653333,-1,0
24414,12941,NaN,NaN,NaN,NaN,NaN,74.589004,"LINESTRING (860185.722 5633386.276, 860185.743...",1894,Lungo Po Alessandro Antonelli,residential,27.653333,-1,0
24415,12941,NaN,NaN,NaN,NaN,NaN,74.589004,"LINESTRING (860185.722 5633386.276, 860185.743...",1906,Lungo Po Alessandro Antonelli,residential,167.233333,-1,0


In [389]:
# Create a new dictionary to store the updated data.
dic_str_data = {}
# Define the relevant columns to store
rel_col  =[col for col in df.columns if col.endswith("right")]+['geometry']
rel_col.remove('index_right')

def return_street_name(group):
    """
    1. "Count the occurrences of polylines with the same name within each group."
    2. "Return the street if a group contains only one unique street name."
    3. "If a single street name predominates within a group, return that name."
    4. "For groups with multiple names, perform a buffer calculation around the respective polylines and determine the largest overlapping area, returning the name associated with that area."
    :param group:
    :return:
    """
    count_names = group['name_right'] .value_counts().sort_values(ascending=False)
    if len(count_names)==1:
        # there is only one name
        my_data = group.iloc[0]
    elif count_names[1]- count_names[0]>1:
        # The highest number of polylines with the same name are bigger at least in 2:
        my_data = group[group['name_right'] ==count_names.index[0]].iloc[0]
    else:
        # otherwise filter those with the most popular name or close to (-1)
        print(group['old_index'].iloc[0])
        str_to_wrk_on  =group[group['name_right'].isin(count_names[count_names- count_names[0]<2].index)]
        buffer_0 = GeoDataFrame(geometry=[str_to_wrk_on.iloc[0]['geometry'].buffer(distance  = 20, cap_style=2)],crs=project_crs) # Buffer around the polyline without name

        streets_right_geo = new_gpd[new_gpd.index.isin(str_to_wrk_on['index_right'])].reset_index() # Get all the applicants polylines and create buffer around
        buffer_1 =GeoDataFrame(geometry=streets_right_geo.buffer(distance  = 20, cap_style=2))
        streets_right_geo['area'] =gpd.overlay(buffer_1, buffer_0, how='intersection').area
        groupy = streets_right_geo.groupby('name')
        my_data_0 = groupy.get_group(groupy.sum()['area'].sort_values(ascending=False).index[0]).sort_values(by= 'area',ascending=False).iloc[0]
        # Get back to the group and find the relevant row by comparing index
        my_data = group[group['index_right'] ==my_data_0['index']].iloc[0]
    # Populate the new dictionary with relevant data
    dic_str_data[my_data['old_index']] = my_data[rel_col].to_list()
_ =df.groupby(old_index).apply(return_street_name)
# convert the dictionary into a dataframe.
updated_df = GeoDataFrame(index=dic_str_data.keys(), data= dic_str_data.values(),columns=[x.replace('_right', '',) for x in rel_col],crs=project_crs)
updated_df['length'] = updated_df.length
new_df = df3.append(updated_df)

6369
6370
6374
6375
6376
6379
6381
6383
6384
6390
6391
6392
6393
6394
6396
6397
6398
6401
6402
6403
6405
6406
6407
6410
6411
6412
6413
6414
6415
6419
6421
6422
6426
6427
6429
6430
6431
6432
6433
6434
6437
6438
6439
6440
6442
6443
6446
6447
6448
6449
6450
6451
6452
6453
6454
6456
6457
6458
6459
6460
6461
6464
6465
6466
6467
6468
6469
6470
6471
6472
6473
6474
6475
6476
6477
6478
6479
6482
6483
6485
6486
6487
6488
6489
6490
6491
6493
6495
6497
6498
6499
6500
6501
6503
6504
6505
6506
6507
6509
6510
6511
6512
6513
6514
6515
6516
6517
6518
6519
6521
6523
6524
6525
6526
6528
6529
6530
6531
6532
6533
6534
6535
6536
6538
6539
6540
6541
6542
6543
6545
6547
6549
6551
6552
6553
6554
6555
6556
6557
6558
6559
6560
6561
6562
6563
6564
6565
6568
6569
6570
6571
6572
6574
6575
6576
6577
6578
6581
6582
6584
6585
6586
6587
6588
6589
6591
6592
6595
6596
6602
6603
6604
6605
6606
6608
6609
6610
6613
6614
6615
6616
6617
6620
6621
6622
6623
6624
6625
6628
6629
6630
6631
6632
6633
6634
6636
6637
6638
6639
6642


In [391]:
new_df.reset_index().to_file(f'{data_folder}/delete_2_nodes/all_new.shp')

In [356]:
t1 = set(new_df['name'].sort_values())
t2 = set(new_gpd['name'].sort_values())
t1==t2

False

In [355]:
new_df['name'].to_list()

['11',
 '11',
 '18',
 '600',
 'Arsenale n°2010',
 'Bertola n°1642',
 'Ciclabile Carducci - Biglieri',
 'Ciclabile Carducci - Biglieri',
 'Ciclabile Carducci - Biglieri',
 'Ciclabile Carducci - Biglieri',
 'Corso Achille Mario Dogliotti',
 'Corso Achille Mario Dogliotti',
 'Corso Adriatico',
 'Corso Adriatico',
 'Corso Alberto Picco',
 'Corso Alberto Picco',
 'Corso Alberto Picco',
 'Corso Alcide De Gasperi',
 'Corso Alcide De Gasperi',
 'Corso Alcide De Gasperi',
 'Corso Alcide De Gasperi',
 'Corso Alcide De Gasperi',
 'Corso Alcide De Gasperi',
 'Corso Alcide De Gasperi',
 'Corso Alessandro Tassoni',
 'Corso Appio Claudio',
 'Corso Appio Claudio',
 'Corso Appio Claudio',
 'Corso Belgio',
 'Corso Belgio',
 'Corso Belgio',
 'Corso Benedetto Brin',
 'Corso Benedetto Croce',
 'Corso Benedetto Croce',
 'Corso Benedetto Croce',
 'Corso Bernardino Telesio',
 'Corso Bernardino Telesio',
 'Corso Bernardino Telesio',
 'Corso Bramante',
 'Corso Bramante',
 'Corso Bramante',
 'Corso Brescia',
 'C

In [358]:
[print(name) for name in  org_network['name'].to_list() if name not in new_df['name'].to_list()]

Corso Giuseppe Siccardi
Corso Monte Grappa
Corso Monte Grappa
Corso Monte Grappa
Corso Monte Grappa
Corso Tassoni
Largo Brescia
Largo Brescia
Largo Toscana
Largo Toscana
Largo Toscana
Largo Toscana
Largo Toscana
Largo Toscana
Passerella del Parco Pietro Colletta
Passerella di Via Mondovì
Piazza Asmara
Piazzale Monte dei Cappuccini
Piazzale Timavo
Ponte Balbis
Ponte Bologna
Ponte Duca degli Abruzzi
Ponte Emanuele Filiberto
Ponte Emanuele Filiberto
Ponte Emanuele Filiberto
Ponte Principessa Isabella di Savoia
Ponte Sassi
Ponte Sassi
Ponte Sassi
Salita Plava
Strada Cunioli Alti
Strada Ospedale San Vito
Strada Ospedale San Vito
Strada dell'Arrivore
Strada dell'Arrivore
Strada dell'Arrivore
Strada dell'Arrivore
Strada dell'Arrivore
Via Ada Marchesini Gobetti
Via Ada Marchesini Gobetti
Via Alfredo Catalani
Via Andrea Pozzo
Via Benaco
Via Camogli
Via Caravaggio
Via Caselle
Via Caselle
Via Caselle
Via Crimea
Via Edoardo Perroncito
Via Edoardo Perroncito
Via Edoardo Perroncito
Via Edoardo Perro

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [351]:
set(new_df['name'])

{'Via Tepice',
 'Via Bordighera',
 'Via Giuseppe Regaldi',
 'Corso Cadore',
 'Largo Saluzzo',
 'Corso Moncalieri',
 'Via Valperga Caluso',
 'Via Antonio Fogazzaro',
 'Via Nomaglio',
 'Via Raffaello Lambruschini',
 'Via Millefonti',
 "Corso Unità d'Italia",
 'Piazza Riccardo Cattaneo',
 'Via Pietro Giuria',
 "Via Alfredo d'Andrade",
 "['Via Giuseppe Sirtori', 'Via Andrea Paris']",
 'Piazza Giovanni Ventitreesimo',
 'Strada della Viola',
 'Strada Antica di Revigliasco',
 'Via Carema',
 'Via Giovanni Antonio Bazzi',
 'Strada Bellacomba',
 "['Via Rapallo', 'Via Mario Pagano']",
 'Via Tommaso Salvini',
 'Via Montello',
 'Strada del Macallè',
 'Via Carlo Cipolla',
 'Via Scalenghe',
 'Ponte Unione Europea',
 'Piazzale Gian Maria Volontè',
 'Via Pinasca',
 'Via Treviso',
 'Via del Carmine',
 'Piazza Enrico Toti',
 'Via Antonio Genovesi',
 'Via Pozzo Strada',
 'Via Tarvisio',
 'Strada del Salino',
 'Via Santa Chiara',
 "['Via Andrea Pozzo', 'Via Pier Luigi Nervi']",
 'Via Domenico Morelli',
 'V

In [ ]:
group = df.groupby(old_index).get_group(9074)
count_names = group['name_right'] .value_counts().sort_values(ascending=False)
count_names

In [341]:
group[group['name_right'].isin(count_names[count_names- count_names[0]<2].index)]
# group[group['name_right'].isin(count_names[count_names==count_names[0]].index)]

,old_index,name_left,highway_left,bearing_left,group_left,is_simplif_left,length_left,geometry,index_right,name_right,highway_right,bearing_right,group_right,is_simplif_right,length_right
8975,9074,NaN,NaN,NaN,NaN,NaN,125.602326,"LINESTRING (855531.798 5638782.755, 855540.759...",4757,Via Ala di Stura,tertiary,3.866667,0,1,1114.781498
14256,9074,NaN,NaN,NaN,NaN,NaN,125.602326,"LINESTRING (855531.798 5638782.755, 855540.759...",4761,Via Ala di Stura,tertiary,174.833333,-1,0,62.701069
14259,9074,NaN,NaN,NaN,NaN,NaN,125.602326,"LINESTRING (855531.798 5638782.755, 855540.759...",9972,Via Guglielmo Reiss Romoli,secondary,113.994737,-1,0,64.724598
14262,9074,NaN,NaN,NaN,NaN,NaN,125.602326,"LINESTRING (855531.798 5638782.755, 855540.759...",9991,Via Guglielmo Reiss Romoli,secondary,94.427273,-1,0,48.291845
14268,9074,NaN,NaN,NaN,NaN,NaN,125.602326,"LINESTRING (855531.798 5638782.755, 855540.759...",9986,Via Guglielmo Reiss Romoli,unclassified,7.050000,-1,0,49.247177
15758,9074,NaN,NaN,NaN,NaN,NaN,125.602326,"LINESTRING (855531.798 5638782.755, 855540.759...",9973,Via Guglielmo Reiss Romoli,secondary,113.994737,-1,0,232.939313
15760,9074,NaN,NaN,NaN,NaN,NaN,125.602326,"LINESTRING (855531.798 5638782.755, 855540.759...",4755,Via Ala di Stura,tertiary_link,56.100000,-1,0,62.901257


In [340]:
count_names[count_names-count_names[0]<2]

Via Guglielmo Reiss Romoli    4
Via Ala di Stura              3
Name: name_right, dtype: int64

In [392]:
updated_df.groupby('name').count().sort_values(by='length',ascending=False)

,highway,bearing,group,is_simplif,geometry,length
name,,,,,,
Corso Vercelli,45,45,45,45,45,45
Corso Regina Margherita,34,34,34,34,34,34
Corso Massimo D'Azeglio,29,29,29,29,29,29
Corso Novara,25,25,25,25,25,25
Strada Castello di Mirafiori,24,24,24,24,24,24
...,...,...,...,...,...,...
Via Filippo Acciarini,1,1,1,1,1,1
Via Finalmarina,1,1,1,1,1,1
Via Fossano,1,1,1,1,1,1


In [ ]:
def delete_false_intersection(self,name_to_splt='name'):

        if 'length' in self.my_network.columns: # To run the code smoothly we need to remove 'length' col if exist
            self.my_network.drop(columns='length',inplace= True)
        org_network= self.my_network.copy()
        # It should be executed twice in order to clean all
        for _  in range(2):
            # First clean all the false node
            self.my_network = remove_false_nodes(self.my_network)
            # the previous function has changed the topology so the length should be updated
            self.my_network['length'] =self.my_network.length
            self.my_network  =self.my_network.drop_duplicates(subset='length') # remove false intersection duplicate many polyline which should be removed
            self.my_network.reset_index(drop=True,inplace= True) # Changes has been made to the geometry so the index should be reset

        # Update street when it is empty
        def update_street_name():
            """
            The function: remove_false_nodes delete the polyline street names so this method repair that by found for those without name the closet polyline from the original dataframe and use its name.
            :return:
            """
            df1= self.my_network
            # Split df1 into two GeoDataFrames: df3 (with names) and df4 (without names)
            df3 = df1[df1[name_to_splt].notna()]
            df4 = df1[df1[name_to_splt].isna()]
            df = gpd.sjoin_nearest(df4,org_network)
            # use only one polyline from the original dataframe for name even if the algorithm may found more
            df = df.reset_index().drop_duplicates(subset='index', keep='first').set_index('index',drop=True)
            rel_col = [col for col in df.columns if col.endswith("right")] + ['length','geometry']
            df =df [rel_col]
            # Replace '_right' with an empty string for all column names
            df.columns = df.columns.str.replace(r'_right$', '', regex=True)
            df = df.drop(columns='index')
            df.index.name = None
            self.my_network =  df.append(df3)

        update_street_name()


In [11]:
obj_intersection.delete_false_intersection()

obj_intersection.my_network.to_file(f'{data_folder}/delete_2_nodes/top_0_0.shp')
obj_intersection.intersection_network()
obj_intersection.my_network.to_file(f'{data_folder}/split_tp_intersection/top_0.shp')

100%|██████████| 1543/1543 [00:08<00:00, 178.94it/s]


In [77]:

line_name ='line_name'
if is_junction:

    exist_data= obj_intersection.my_network.reset_index().reset_index(names=line_name)
    my_roundabout=Roundabout(exist_data)
    deadend_lines, deadend_pnts = my_roundabout.deadend()

    # update the current network
    change_geo = my_roundabout.my_spatial_join(deadend_lines, deadend_pnts,line_name)
    my_roundabout.update_the_current_network(change_geo)

    my_roundabout.network.drop_duplicates(subset=line_name,inplace=True)
    # Improve roundabout
    # First buffer around centroid
    centr_name= 'centr_name'
    buffer_around_centroid= my_roundabout.centroid['geometry'].buffer(cap_style=1, distance=30)

    # s_join between buffer to lines (reset index to retain the original centroid name which can apper more than one in the results). always stay with data you need and with understandable name
    roundabout_with_lines =gpd.sjoin(left_df=GeoDataFrame(geometry=buffer_around_centroid,crs=project_crs).reset_index(),right_df=my_roundabout.network[['geometry',line_name]]).drop_duplicates(subset=['index',line_name]).rename(columns={"index":centr_name})[['geometry',line_name,centr_name]]

    # To facilitate the searching process
    my_roundabout.network.set_index(line_name,inplace=True)
    # To facilitate easy access to point centroid geometry data, it is advisable to store the information in an object that provides efficient retrieval.
    pnt_centroid_temp = my_roundabout.centroid['geometry']
    #  Group the data by centroid
    for center_line in roundabout_with_lines.groupby(centr_name):
        #  Iterate over each group after performing a groupby() operation
        for center in center_line[1].itertuples():
            # Find the line that connects to the current centroid and obtain its vertices
            line_to_test = my_roundabout.network.loc[center[2]]
            vertices_line = list(line_to_test['geometry'].coords)
            pnt_test = [vertices_line[0],vertices_line[-1]]
            # To determine if the current line is already connected to the current centroid,.
            is_connected = my_roundabout.centroid[my_roundabout.centroid['geometry'].isin([Point(pnt_test[0]),Point(pnt_test[-1])])]
            if len(is_connected)>0 and center[3] in is_connected['name']:
                continue

            if len(vertices_line)==2:
                vertices_line.insert(1, pnt_centroid_temp[center[3]])
            else:
                my_list = [pnt_centroid_temp[center[3]].distance(Point(temp)) for temp in vertices_line]
                # Find the minimum index
                min_index = min(range(len(my_list)), key=my_list.__getitem__)
                if min_index ==0:
                    vertices_line.insert(0,pnt_centroid_temp[center[3]])
                elif min_index == len(my_list)-1:
                    vertices_line.append(pnt_centroid_temp[center[3]])
                else:
                    vertices_line[min_index] = pnt_centroid_temp[center[3]]
            new_geo = LineString(vertices_line)
            my_roundabout.network.at[center[2],'geometry'] = new_geo
# Extend
    new_network2 = my_roundabout.network.reset_index()
    new_network2.drop(columns='index',inplace=True)
else:
    new_network2=  obj_intersection.my_network.reset_index()

new_network2.to_file(f'{data_folder}/ra_network.shp')


In [78]:
extend_lines_f= extend_lines(new_network2,100)
extend_lines_f['length'] = extend_lines_f.length
extend_lines_f.to_file(f'{data_folder}/extend_network.shp')

obj_intersection_1 = Intersection(extend_lines_f.copy(),1)
obj_intersection_1.delete_false_intersection('str_name')
obj_intersection_1.my_network.to_file(f'{data_folder}/delete_2_nodes/top_20.shp')

In [79]:
obj_intersection_1.intersection_network()

 91%|█████████ | 49758/54734 [00:07<00:00, 6464.56it/s]

6409,11000:GEOMETRYCOLLECTION (LINESTRING (849247.5855611513 5630512.478084865, 849222.2137190597 5630513.776590068), POINT (849247.096878846 5630514.638943701))
11000,6409:GEOMETRYCOLLECTION (LINESTRING (849222.2137190597 5630513.776590068, 849247.5855611513 5630512.478084865), POINT (849247.096878846 5630514.638943701))


100%|██████████| 945/945 [00:04<00:00, 195.26it/s]


In [80]:
# Clear short segments
final2 = EnvEntity(obj_intersection_1.my_network.reset_index())
final2.update_the_current_network(final2.get_deadend_gdf(delete_short=100))
final2.network.to_file(f'{data_folder}/final2.shp')